In [1]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2025-06-08 23:22:41--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8002::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4014 (3.9K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.92K  --.-KB/s    in 0s      

2025-06-08 23:22:41 (29.7 MB/s) - ‘minsearch.py’ saved [4014/4014]



In [1]:
import requests 
import minsearch
import json 
import os 
from groq import Groq 
from dotenv import load_dotenv 
load_dotenv()

client = Groq(api_key = os.environ.get('GROQ_API_KEY'))

# docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
file_path = '/Users/pawanchahar/LLM_ZoomCamp/01-intro/documents.json'

with open(file_path , 'rt') as f_in:
    documents_raw = json.load(f_in)


# docs_response = requests.get(docs_url)
# documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

/Users/pawanchahar/LLM_ZoomCamp/open-source-llm/minsearch.py:9: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [2]:
query = 'the course already started. Can I still enroll?'

In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [4]:
search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [5]:
def build_prompt(query, search_results):
    prompt_template = """ 
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.


QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt 

In [6]:
def llm(prompt):
    response = client.chat.completions.create(
        model= 'llama3-70b-8192',
        messages= [{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [7]:
query = 'the course already started. Can I still enroll?'


def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [8]:
llm('write that this is a test')

'**THIS IS A TEST'

In [9]:
print(_)

**THIS IS A TEST
